In [26]:
!module load cuda

/bin/bash: line 1: module: command not found


In [27]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.transforms as T

import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from jupyterplot import ProgressPlot
from tqdm import tqdm, trange


In [28]:
from datetime import datetime, timedelta

In [29]:
class WeatherDataset(Dataset):
    def __init__(self, weather_data = 'data/big_arpafvg.csv', img_dir = 'data/cut_images'):
        initial_data=pd.read_csv(weather_data)
        initial_data = initial_data.drop(columns=['Temp. min gradi C','Temp. med gradi C','Temp. max gradi C','Vento med km/h','Dir. V. max gradi N'])
        #normalize the data
        for col in initial_data.columns:
            if col != 'giorno' and col != 'mese' and col != 'anno':
                initial_data[col] = (initial_data[col] - initial_data[col].mean()) / initial_data[col].std()
        self.weather_data = initial_data
        self.img_dir = img_dir
        self.seq_length = 7
        self.target_column_index = 3
        self.date_generated = []

    def __getitem__(self, date):
        #get day month year from date in format dd-mm-yyyy
        day, month, year = date.split('_')
        #get from weather data the row with the same date
        weather_data = self.weather_data[(self.weather_data['giorno'] == int(day)) & (self.weather_data['mese'] == int(month)) & (self.weather_data['anno'] == int(year))]
        weather_data = torch.tensor(weather_data.values[0])

        image = os.path.join(self.img_dir, str(date) + '.jpg')  
        image = read_image(image)   
        return [image, weather_data]
    
    def __len__(self):
        return len(self.weather_data)
    
    def date_generation(self, start_date, end_date):
        start_date = datetime.strptime(start_date, "%d_%m_%Y")
        end_date = datetime.strptime(end_date, "%d_%m_%Y")
        date_generated = [start_date + timedelta(days=x) for x in range(0, (end_date-start_date).days)]
        #transform the list of datetime objects in a list of strings in format dd-mm-yyyy
        date_generated = [date.strftime("%d_%m_%Y") for date in date_generated]
        self.date_generated=date_generated
        return date_generated

    def date_generation_multiple(self, start_date1, end_date1, start_date2, end_date2, start_date3, end_date3):
        start_date1 = datetime.strptime(start_date1, "%d_%m_%Y")
        end_date1 = datetime.strptime(end_date1, "%d_%m_%Y")
        start_date2 = datetime.strptime(start_date2, "%d_%m_%Y")
        end_date2 = datetime.strptime(end_date2, "%d_%m_%Y")
        start_date3 = datetime.strptime(start_date3, "%d_%m_%Y")
        end_date3 = datetime.strptime(end_date3, "%d_%m_%Y")
        date_generated1 = [start_date1 + timedelta(days=x) for x in range(0, (end_date1-start_date1).days)]
        date_generated2 = [start_date2 + timedelta(days=x) for x in range(0, (end_date2-start_date2).days)]
        date_generated3 = [start_date3 + timedelta(days=x) for x in range(0, (end_date3-start_date3).days)]
        date_generated = date_generated1 + date_generated2 + date_generated3
        #transform the list of datetime objects in a list of strings in format dd-mm-yyyy
        date_generated = [date.strftime("%d_%m_%Y") for date in date_generated]
        self.date_generated=date_generated
        return date_generated
    
    def create_sequence(self, date, len_seq):
        batch_xs =[]
        batch_imgs = []
        batch_ys = []
        for elem in date:
            ys = []
            end_of_week = elem.split('_')
            end_of_week = datetime(int(end_of_week[2]), int(end_of_week[1]), int(end_of_week[0])) + timedelta(days = len_seq)
            end_of_week = end_of_week.strftime("%d_%m_%Y")
            imgs, xs =[], []
            week=self.date_generation(elem,end_of_week)
            for day in week:
                a , b = self.__getitem__(day)
                imgs.append(a)
                xs.append(b)


            next = end_of_week.split('_')
            next = datetime(int(next[2]), int(next[1]), int(next[0])) + timedelta(days = 1)
            next = next.strftime("%d_%m_%Y")
            ys.append(self.__getitem__(next)[1][self.target_column_index])
            imgs = np.array(imgs)
            xs = np.array(xs)
            ys = np.array(ys)
            imgs = torch.from_numpy(imgs).float()
            xs = torch.from_numpy(xs).float()
            ys = torch.from_numpy(ys).float()
        batch_xs.append(xs)
        batch_imgs.append(imgs)
        batch_ys.append(ys)
                    
        batch_xs = np.array(batch_xs)
        batch_imgs = np.array(batch_imgs)
        batch_ys = np.array(batch_ys)
        batch_xs = torch.from_numpy(batch_xs).float()
        batch_imgs = torch.from_numpy(batch_imgs).float()
        batch_ys = torch.from_numpy(batch_ys).float()
        return batch_imgs, batch_xs, batch_ys

In [30]:
dataset = WeatherDataset()

In [31]:
#generate a list of strings dd-mm-yyyy from 01-06-2023 to 15-6-2024
#from datetime import datetime, timedelta
#start_date = datetime.strptime("20_12_2023", "%d_%m_%Y")
#end_date = datetime.strptime("20_03_2024", "%d_%m_%Y")
#date_generated = [start_date + timedelta(days=x) for x in range(0, (end_date-start_date).days)]
#transform the list of datetime objects in a list of strings in format dd-mm-yyyy
#date_generated = [date.strftime("%d_%m_%Y") for date in date_generated]


In [ ]:
dataset.date_generation_multiple("01_01_2022", "20_03_2022", "01_01_2023", "20_03_2023", "01_01_2024", "20_03_2024")
date_generated = dataset.date_generated

KeyboardInterrupt: 

In [ ]:
train_len = int(0.8 * len(date_generated))
test_len = len(date_generated) - train_len
#split the dataset: date_trainset contains the first 80% of the dates, date_testset contains the remaining 20%
date_trainset = date_generated[:train_len]
date_testset = date_generated[train_len:]

In [ ]:
#MODIFIED CNN

import torch
import torch.nn as nn
import torch.nn.functional as F

class DeepWeather(nn.Module):
    def __init__(self):
        super(DeepWeather, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=(3, 3), stride=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(p=0.3)
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), stride=1)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), stride=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), stride=1)
        self.bn5 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), stride=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(6154, 1024)
        self.fc2 = nn.Linear(1024, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x1, x2):
        # Reshape x1 to [batch_size, channels, height, width]
        x1 = x1.view(x1.size(0), x1.size(2), x1.size(3), x1.size(4))
        
        x1 = self.bn1(self.dropout(self.pool(F.leaky_relu(self.conv1(x1)))))
        x1 = self.bn2(self.dropout(self.pool(F.leaky_relu(self.conv2(x1)))))
        x1 = self.bn3(self.dropout(self.pool(F.leaky_relu(self.conv3(x1)))))
        x1 = self.bn4(self.dropout(self.pool(F.leaky_relu(self.conv4(x1)))))
        x1 = self.bn5(self.dropout(self.pool(F.leaky_relu(self.conv5(x1)))))
        x1 = self.bn6(self.dropout(self.pool(F.leaky_relu(self.conv6(x1)))))
        x1 = torch.flatten(x1, start_dim=1)

        # Ensure x2 has the same number of dimensions as x1
        if x2.dim() == 3:
            x2 = x2.squeeze(1)

        x = torch.cat((x1, x2), dim=1)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [ ]:
#ORIGINAL CNN
'''
class DeepWeather(nn.Module):
    def __init__(self):
        super(DeepWeather, self).__init__()

        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = (3, 3), stride = 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(p = 0.3)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = (3, 3), stride = 2)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = (3, 3), stride = 2)
        self.bn3 = nn.BatchNorm2d(128)
        self.fc1 = nn.Linear(6282, 1024)
        self.fc2 = nn.Linear(1024, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, inputs):
        x1, x2 = inputs[0], inputs[1]
        

        x1 = self.bn1(self.dropout(self.pool(F.leaky_relu(self.conv1(x1)))))
        x1 = self.bn2(self.dropout(self.pool(F.leaky_relu(self.conv2(x1)))))
        x1 = self.bn3(self.dropout(self.pool(F.leaky_relu(self.conv3(x1)))))
        x1 = torch.flatten(x1, start_dim = 1)

        x = torch.cat((x1, x2), dim = 1)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)

        return x
'''

'\nclass DeepWeather(nn.Module):\n    def __init__(self):\n        super(DeepWeather, self).__init__()\n\n        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = (3, 3), stride = 2)\n        self.pool = nn.MaxPool2d(2, 2)\n        self.dropout = nn.Dropout(p = 0.3)\n        self.bn1 = nn.BatchNorm2d(32)\n        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = (3, 3), stride = 2)\n        self.bn2 = nn.BatchNorm2d(64)\n        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = (3, 3), stride = 2)\n        self.bn3 = nn.BatchNorm2d(128)\n        self.fc1 = nn.Linear(6282, 1024)\n        self.fc2 = nn.Linear(1024, 128)\n        self.fc3 = nn.Linear(128, 1)\n\n    def forward(self, inputs):\n        x1, x2 = inputs[0], inputs[1]\n        \n\n        x1 = self.bn1(self.dropout(self.pool(F.leaky_relu(self.conv1(x1)))))\n        x1 = self.bn2(self.dropout(self.pool(F.leaky_relu(self.conv2(x1)))))\n        x1 = self.bn3

In [ ]:
def train(model, dataset, train_loader, test_loader, criterion, optimizer, epochs = 50, first_time = True, num_saved_epochs = 0):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model = model.to(device)    
    
    train_loss_epochs = []
    test_loss_epochs = []
    bar = trange(epochs, desc=f"Epoch ?/?, Train Loss: ?, Test Loss: ?")
    for epoch in bar:
        model.train()
        train_losses = []
            
        for date in train_loader:
            imgs, xs, ys = dataset.create_sequence(date, 1)
            imgs = imgs.to(device)
            xs = xs.to(device)
            ys = ys.to(device)

            optimizer.zero_grad()
            outputs = model(imgs, xs)
            loss = criterion(outputs, ys)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            
        train_loss_epochs.append(np.mean(train_losses))
        model.eval()
        test_losses = []
        for date in train_loader:
            
            imgs, xs, ys = dataset.create_sequence(date, 1)
            imgs = imgs.to(device)
            xs = xs.to(device)
            ys = ys.to(device)
            outputs = model(imgs, xs)
            loss = criterion(outputs.squeeze(), ys)
            test_losses.append(loss.item())
                
        test_loss_epochs.append(np.mean(test_losses))
        bar.set_description(f"Epoch {epoch + 1}/{epochs}, Train Loss: {np.mean(train_losses)}, Test Loss: {np.mean(test_losses)}")
        #if first_time:
            #torch.save(model.state_dict(), f"deepweather_epoch{num_saved_epochs + epoch + 1}.pth")
    return train_loss_epochs, test_loss_epochs

In [ ]:
#Uncomment the lines below if you want to train/load a pretrained model
#num_saved_epochs = 50
#model = model.load_state_dict(torch.load(f'weights/epoch_{num_saved_epochs}'))

#Comment the (ONE) line below if you want to train/load a pretrained model.
#!mkdir weights
model = DeepWeather()
batch_size = 5

date_trainloader = DataLoader(date_trainset, batch_size = batch_size, shuffle = False)

date_testloader = DataLoader(date_testset, batch_size = batch_size, shuffle = False)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 10e-6, weight_decay = 10e-5)

In [ ]:
train_losses, test_losses = train(model, dataset, date_trainloader, date_testloader, criterion, optimizer, epochs = 50, first_time = True, num_saved_epochs = 0)

Epoch ?/?, Train Loss: ?, Test Loss: ?:   0%|          | 0/50 [00:00<?, ?it/s]

torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([

/home/gaialessio/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([

Epoch 1/50, Train Loss: 0.38219295633344147, Test Loss: 0.3408639739684172:   2%|▏         | 1/50 [00:21<17:12, 21.08s/it]

torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([1, 10])
torch.Size([1, 6144])
torch.Size([

Epoch 1/50, Train Loss: 0.38219295633344147, Test Loss: 0.3408639739684172:   2%|▏         | 1/50 [00:41<33:51, 41.45s/it]

torch.Size([1, 6144])
torch.Size([1, 10])


KeyboardInterrupt: 

In [ ]:
train_losses, test_losses = train(model, dataset, batch_size, date_trainloader, date_testloader, criterion, optimizer, epochs = 50, first_time = True, num_saved_epochs = 0)

Epoch ?/?, Train Loss: ?, Test Loss: ?:   0%|          | 0/50 [00:00<?, ?it/s]

date_inputs length: 5, batch_size: 5


AttributeError: 'int' object has no attribute 'split'

In [ ]:
epochs = [epoch for epoch in range(50)]
plt.plot(epochs, train_losses, label = 'Training Loss')
plt.plot(epochs, test_losses, label = 'Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
def predict(model, date):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    inputs = dataset[date]
    inputs = [inputs[0].float().unsqueeze(0).to(device), inputs[1].float().unsqueeze(0).to(device)]

    outputs = model(inputs)    
    
    return outputs

In [ ]:
#print(predict(model, '05_06_2023'))
#print(dataset['05_06_2023'][1][3])

In [ ]:
for date in date_testset:
    predicted_rain = predict(model, date)
    #non normalized results
    predicted_rain = predicted_rain 
    print(f"Predicted rain normalized for {date} is {predicted_rain}")
    #while real rain is
    print(f"Real rain normalized  for {date} is {dataset[date][1][3]}")   
